In [1]:
import pyexcel_xlsx as xl
import pandas as pd
import numpy as np
from collections import OrderedDict

In [2]:
def OtherMaker(path_read, path_write, threshold): # На вход пути считывания и записи а также пороговое значение. Если у партии
                                                  # число парламентариев меньше порога, то ее сливаем в Others.
    
    counData = xl.get_data(path_read)
    
    party_dict = dict() # тут словарик, где у каждой партии будут индикаторы: 1 - если ее не надо сливать в others и 0 если надо
    for key in counData.keys():
        for i in counData[key][0][1:-4]:
            if i not in party_dict.keys():
                party_dict[i] = 0
    
    newOrderedDict = dict() # Сюда будем записывать датафреймы с каждой страницы
    for key in counData.keys():
        df = counData[key]
        df[0] = df[0][1:]
        parties_names = df[0][0:df[0].index('')]
        regions_names = []
        data = []
        for element in df[1:]:
            regions_names += [element[0]]
            data += [element[1:]]
        pd_df = pd.DataFrame(index = regions_names, columns = parties_names, data = data) # Делаем из каждой страницы датафрейм
        for i in pd_df.columns:
            if party_dict[i] != 1:
                indicator = 0 # Индикатор того, что у данной партии нет региона, в котором у нее больше 5%
                for rate in pd_df[i]:
                    if rate >= threshold: # Тут проверка, что нет больше порогового значения
                        indicator = 1
                if indicator == 1:
                    party_dict[i] = 1
                else:
                    party_dict[i] = 0
        newOrderedDict[key] = pd_df
        
    OneMoreDict = dict() # Это еще один промежуточный датафрейм
    for sheet in newOrderedDict:
        df = newOrderedDict[sheet]
        parties_to_other = []
        for i in party_dict.keys():
            if party_dict[i] == 0 and (i in list(df.columns)):
                parties_to_other += [i]
        if parties_to_other != []:
            df['Others'] = np.zeros(df.shape[0]) # Тут в общем сливаем и суммируем
            for i in parties_to_other:
                df['Others'] += np.array(df[i])
                df.drop(i, axis = 1, inplace = True)
        OneMoreDict[sheet] = df
    DictToWrite = OrderedDict(sorted(OneMoreDict.items(), key=lambda t: t[0])) # Специально сделал OrderedDict, чтобы записать 
                                                                               # в файл в нужном порядке
        
    writer = pd.ExcelWriter(path_write, engine='xlsxwriter')
    for i in DictToWrite.keys():
        DictToWrite[i].to_excel(writer, sheet_name = i, startrow = 0)
    writer.save()
    
    return "It is done!"

In [4]:
OtherMaker('banks_test.xlsx', 'banks_test_new.xlsx', 5)

'It is done!'